In [1]:
from datetime import timedelta
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [2]:
ccac_test = pd.read_excel('CCAC_data.xlsx')
ccac_test.head()

,Id,EmailTo,CC,SenderName,SenderEmailAddress,Subject,Body
0,1,Mohdhafi Agdern,NaN,Brandon Aristeo Akcali,AkcaliBra3136@ccac.sales.com,RE: [EXT]Re: December meeting request,Thank you! I just sent a hold time. I will ca...
1,2,Shige Akbatur; Jenny Acord; Christop J Acord; ...,NaN,Michael J Abbott,AbbottMic4370@ccac.sales.com,ccac sales-EE Joint Technology Visioning Sessi...,"Over the past 8 months, System Planning has be..."
2,3,Babur Rais Abolt; Brandon Aristeo Akcali; Osam...,NaN,Suchit Abdullah,AbdullahSuc1912@ccac.sales.com,Canceled: UUDEX Discussion,_________________________________________...
3,4,Hamad A Aivalotis,NaN,Brandon Aristeo Akcali,AkcaliBra3136@ccac.sales.com,check in with Hannah,_______________________________...
4,5,Brandon Aristeo Akcali,NaN,Mohammed Akpobasah,AkpobasahMoh3564@ccac.sales.com,HOLD - IN3/ccac sales Conversation on Cybersec...,"One I confirm a date and time with Julie, I wi..."


In [116]:
ccac_test.columns

Index(['Id', 'EmailTo', 'CC', 'SenderName', 'SenderEmailAddress', 'Subject',
       'Body'],
      dtype='object')

In [118]:
ccac_test['Subject'] = ccac_test['Subject'].apply(str)
ccac_test['Body'] = ccac_test['Body'].apply(str)
ccac_test['all'] = ccac_test['Subject']+" "+ccac_test['Body']

In [2]:
train_dict = {'EmailTo':[],'Sender':[],'Subject':[],'Body':[]}

In [3]:
import mailbox
import csv
def getbody(message): #getting plain text 'email body'
    body = None
    if message.is_multipart():
        for part in message.walk():
            if part.is_multipart():
                for subpart in part.walk():
                    if subpart.get_content_type() == 'text/html':
                        body = subpart.get_payload(decode=True)
            elif part.get_content_type() == 'text/html':
                body = part.get_payload(decode=True)
    elif message.get_content_type() == 'text/html':
        body = message.get_payload(decode=True)
    return body

In [4]:
# data from mbox, all phishing
for m in mailbox.mbox('emails-phishing.mbox'):
    train_dict['EmailTo'].append(m['to'])
    train_dict['Sender'].append(m['from'])
    train_dict['Subject'].append(m['subject'])
    train_dict['Body'].append(getbody(m))

In [5]:
# data from phishing_data
phishing = pd.read_excel('PhishingEmailData.xlsx')


In [6]:
for t,f,s,c in zip(phishing['To'],phishing[ 'Sender_Email '],phishing['Email_Subject'],phishing['Email_Content']):
    train_dict['EmailTo'].append(t)
    train_dict['Sender'].append(f)
    train_dict['Subject'].append(s)
    train_dict['Body'].append(c)

In [7]:
ptrain = pd.DataFrame(train_dict)
ptrain = ptrain.fillna(" ")
ptrain['Subject'] = ptrain['Subject'].apply(str)
ptrain['Body'] = ptrain['Body'].apply(str)
ptrain['all'] = ptrain['Subject'] + ' ' + ptrain['Body']
ptrain['phishing'] = 1
train_1 = ptrain[['all','phishing']]
train_1.head()

,all,phishing
0,IMPORTANT ACCOUNT NOTICE FROM SOUTHTRUST BANK ...,1
1,eBay account verification! b'<DIV><PRE><TT><ST...,1
2,eBay account verification! b'<DIV>\n<DIV>\n<P>...,1
3,KeyBank Customer Confirm Your Identity b'<styl...,1
4,KeyBank Customer Confirm Your Identity b'<styl...,1


In [8]:
train_1.to_csv('phishing_dataset.csv')

In [104]:
train = pd.read_csv("train.csv")
test_csv = pd.read_csv("test.csv")
train = train.fillna(" ")
test_csv = test_csv.fillna(" ")
train['all'] = train['subject'] + ' ' + train['email'] # 合并两个特征

In [111]:
train_2 = train[train['spam']==0][['all']]
train_2['phishing'] = 0
train_2.head()

,all,phishing
0,Subject: A&L Daily to be auctioned in bankrupt...,0
1,"Subject: Wired: ""Stronger ties between ISPs an...",0
3,Subject: liberal defnitions\n Depends on how m...,0
4,Subject: RE: [ILUG] Newbie seeks advice - Suse...,0
5,Subject: Five\n URL: http://diveintomark.org/a...,0


In [113]:
train = pd.concat([train_1,train_2])
len(train)

8677

In [61]:
for m in mailbox.mbox('emails-phishing.mbox'):
    print([m['subject'], m['from'],m['to']])

['IMPORTANT ACCOUNT NOTICE FROM SOUTHTRUST BANK [Thu, 16 Jun 2005 12:20:57 -0600]', 'SouthTrust <identdep_op718@southtrust.com>', 'user@example.com']
['eBay account verification!', '"emailconfirm@ebay.com" <emailconfirm@ebay.com>', None]
['eBay account verification!', 'emailconfirm@ebay.com', None]
['KeyBank Customer Confirm Your Identity', 'Key Team <support@keybank.com>', 'user@example.com']
['KeyBank Customer Confirm Your Identity', 'Key Team <support@keybank.com>', 'user@biochemistry.BIOC.cwru.edu']
['Account compromised: billing information moved or changed.', 'PayPal <acc-validity@paypal.com>', 'undisclosed-recipients: ;']
['Verify your PayPal Account', '"Account Service" <service@paypal.com>', 'undisclosed-recipients: ;']
['Notification of Limited Account Access (Routing Code: PP-091-233-629)', 'Paypal Account Review Department <service@paypal.com>', 'user@biochemistry.BIOC.cwru.edu']
['Notification of Limited Account Access (Routing Code: C840-L001-Q190-T1830) Unauthorized Acce

['Online Kunden', 'maggie chrissy <288emlyn@postbankde.com>', 'user@biochemistry.BIOC.cwru.edu']
['Viza Security Alert', '"Visa Security Dep." <Security@VerifiedByVisa.com>', 'Jose <user@example.com>']
['FPO Notice: Important information about your eBay account', '"eBay Customer Support" <customer@ebay.com>', 'user@biochemistry.BIOC.cwru.edu']
['FPO Notice: Important information about your eBay account', '"eBay Customer Support" <customer@ebay.com>', 'user@example.com']
['Update Information - Verification Required', 'Wells Fargo Online Banking <accounts@wellsfargo.com>', 'user@example.com']
['Update Your PayPal Account !', '"service@paypal.com" <service@paypal.com>', 'user@example.com']
['Postbank Online Banking', 'isto brook <3khayroll@postbankde.com>', 'user@biochemistry.BIOC.cwru.edu']
['EBAY: IMPORTANT NOTIFICATION', 'eBay Inc <support_ref_09205@ebay.com>', 'user@example.com']
['EBAY: IMPORTANT NOTIFICATION', 'eBay Inc <support_ref_09205@ebay.com>', 'user@example.com']
['Your accou

['Accounts maintenance and verifications', '"CustomerSuportForm@CUNA.org" <CustomerSuportForm@CUNA.org>', 'undisclosed-recipients: ;']
['IMPORTANT BANK MAIL FROM EBAY', 'eBay <custservice_ref_400760@ebay.com>', 'user@example.com']
['eBay TKO NOTICE : Your account could be suspended', '"eBay Security Department" <account.access@eBay.com>', 'undisclosed-recipients: ;']
['Central Bank - banking@home e-statements and Online Bill Pay deactivation', '"Central Bank" <custom.service@centralbank.net>', 'undisclosed-recipients: ;']
['Central Bank - banking@home e-statements and Online Bill Pay deactivation', '"Central Bank" <custom.service@centralbank.net>', 'undisclosed-recipients: ;']
['Protect the security of your online account. ', 'C U N A <service@cuna.org>', 'undisclosed-recipients: ;']
['Your PayPal Account Information', 'PayPal <service@paypal.com>', 'user@example.com']
['Question from ebay member', '"aw-confirm@ebay.com" <aw-confirm@ebay.com>', 'undisclosed-recipients: ;']
['Question F

['Important Account Notification', '"Fifth Third Bank 2006" <reference_id_57126451.cust@53.com>', '<joliveira@example.com>']
['Fifth Third Bank Email - [user@example.com] - Verification Mon, 18 Sep 2006 07:24:39 -0500', '"Fifth Third Bank 2006" <support-reference94706362514993id@53.com>', 'user <user@example.com>']
['Important account notice from Fifth Third Bank', '"FIFTH THIRD BANK 2006" <operator_443137581015id@53.com>', 'user <user@example.com>']
['Urgent Security Notice [Wed, 11 Oct 2006 09:20:04 -0100]', '"SunTrust Bank\'06" <onlinesupport_id-178350827569id@suntrust.com>', 'user <user@example.com>']
['KeyBank Reminder: Please Update Your Data', '"KEYBank" <custsupport_7729087377id@key.com>', 'user <user@example.com>']
['Fifth Third Bank: Account Investigation Warning', "Fifth Third Bank'06 <custsupport_13312@53.com>", 'user@example.com']
['Important Banking MaiI', "Fifth Third Bank'06 <support_reference8137@53.com>", 'user@example.com']
['Fifth Third Bank: Service Message 1 Novem

['Bank of America - account notification!', '"Bank of America" <manager_id5017585ib@bankofamerica.com>', 'user <user@example.com>']
['0fficial Information For Client Of Fifth Third Bank -Sun, 28 Jan 2007 18:03:45 -0800', '"Fifth Third Bank" <refnumber-num801672ver@security.53.com>', 'user <user@example.com>']
['Fifth Third Bank - service message!', '"Fifth Third Bank" <manager_id379105303ib@53.com>', 'user <user@example.com>']
['Bank of America - account notification. [Tue, 30 Jan 2007 13:59:51 -0800]', '"Bank of America" <online-1518486240ib@bankofamerica.com>', 'user <user@example.com>']
['Fifth Third Bank Account Secure Confirmation Procedure -Wed, 31 Jan 2007 23:24:38 -0800', '"Fifth Third Bank" <clientservice-8958618607ver@security.53.com>', 'user <user@example.com>']
['BB&T reminder: please update your details. -Fri, 02 Feb 2007 13:30:57 -0800', '"Branch Banking and Trust" <clientservice_id111646ib@bbt.com>', 'user <user@example.com>']
['Fifth Third Bank reminder: please update y

...

['Estimado usuario de Online Banking !', 'Gary <Kris@nitches.com>', 'user@example.com']
['Capital One Bank Notification - Please Read - ID: COB495886838', '"Capital One Bank"<updating@capitalone.com>', 'undisclosed-recipients:;']
['You have added a new e-mail address for your PayPal account.', '"Paypal" <admin@paypal.com>', 'undisclosed-recipients: ;']
['Status', 'smith@serverendp.ssh', 'user@example.com']
['Please upDate your account - Personal information erorror !', '"service@BankofAmerica"<service@security>', 'user@example.com']
['Security: Please verify your PayPal Account!', '"PayPal Security"<security@paypal.com>', 'undisclosed-recipients:;']
['Verify your details with BB&T -Sun, 08 Apr 2007 07:20:50 -0500', '"Branch Banking and Trust" <investigation-id86216ib@bbt.com>', 'user <user@example.com>']
['Notification: Your account had been recently compromised.', '"Guaranty Bond Bank"<member@anti-fraud.com>', 'undisclosed-recipients:;']
['Customer Service: Your Account In BB&T! [Mon,

['Question about item - Respond now!', '"eBay" <message@ebay.com>', 'madonna-digest@example.com, user@example.com']
['Please visit the resolution center', '"PayPal"<service@paypal.com>', 'undisclosed-recipients:;']
['Account Information Update', '"PayPal"<service@paypal.com>', 'undisclosed-recipients:;']
['BB&T customer service: instructions for customer!', '"Branch Banking and Trust" <clientservice.ref10188240.cm@bbt.com>', 'user <user@example.com>']
['New email address added to your account!', 'service@paypal.com<accounts@paypal.com>', 'undisclosed-recipients:;']
['BB&T: important information!', '"BB&T" <clientservice.refTP77566297111UU.cm@bbt.com>', 'user <user@example.com>']
['Resolution center', '"PayPal" <service@paypal.com>', 'undisclosed-recipients: ;']
['Please Update Your Profile - Personal Information Error', 'Bank of America Service <service101@email-bankofamerica.com>', 'user@example.com']
['Please restore your account access', '"service@paypal.com"<service@paypal.com>', '

['New security system Del Norte Credit Union !', '"Del Norte Credit Union"<security@dncu.org.>', 'undisclosed-recipients:;']
['Question from focusonthenow', '"eBay Inc." <messages@ebay.com>', 'user@example.com']
['Notizia', '"info@poste.it" <info@poste.it>', 'user@example.com']
['Regions Bank: account secure confirmation! (message id: m0491402bl)', '"Regions bank" <clientcare.refpj02239803851289.cm@regions.com>', 'user <user@example.com>']
['Question about Item -- Respond Now ', '"eBay"<messages@ebay.com>', 'undisclosed-recipients:;']
["Congratulations!You're an eBay Silver PowerSeller .Get your PowerSeller benefits now!", '"eBay PowerSellers !"<eBay@powerseller.com>', 'undisclosed-recipients:;']
['>Question for item #250026510836 - Respond Now<', '"eBay.lnc."<aw-tech3@ebay.com>', 'undisclosed-recipients:;']
['Online Banking Urgent Security Notice', 'Halifax Bank Plc <onlinebanking@halifax.co.uk>', 'user@example.com']
['eBay Unpaid Item Mutual Agreement for Item #2070651641 - Response 

['TKO NOTICE: Suspicious Activity.', '"Ebay" <aw-confirm@ebay.com>', 'undisclosed-recipients: ;']
['TKO NOTICE: Suspicious Activity.', '"Ebay"<aw-confirm@ebay.com>', 'undisclosed-recipients:;']
['Bank of the West customer service informs you! (message id: Y77098333369)', '"Bank of the West" <clientservice.reft163830247345.bow@bankofthewest.com>', 'user <user@example.com>']
['Question from eBay member - Respond Now!!!', '"eBay" <member@ebay.com>', 'kra@example.com, user@example.com']
['Message from eBay Member Regarding Item #180114826067', '"eBay  Member"<member@ebay.com>', 'undisclosed-recipients:;']
['Capital One Bank customer service: urgent notification! (mess_id: R48946156616PP)', '"Capital One Bank" <clientdepmnt.refLG97989011Q.tn@capitalonebank.com>', 'user <user@example.com>']
['official information for client of Bank of the West! (mess_id: rf7287063)', '"Bank of the West" <clientdepmnt.refD44506676525.bow@bankofthewest.com>', 'user <user@example.com>']
['Question from eBay Mem

['Account Secure Confirmation!', '"Citizens Bank and Charter One Bank" <clientserviceteam.refy5004295a.gps@citizensbank.com>', '"Umeet" <umeet@uninet.edu>']
['eBay: confirm your online account details! (mess_id: 24685965526)', '"eBay" <customerdepmnt.refT867758445204.nf@ebay.com>', 'user <user@example.com>']
['Citizens Bank and Charter One Bank customer service: urgent security notice! (message id: ND0298497CY)', '"Citizens Bank and Charter One Bank" <clientdepmnt.refr36767321.gps@citizensbank.com>', 'user <user@example.com>']
['Message from eBay member ', '"eBay member:linda625" <eBay_sellers@eBay_messages.com>', 'undisclosed-recipients: ;']
['Citizens Bank and Charter One Bank customer service:\n\tcustomer details confirmation! (mess_id: e79601692q)', '"Citizens Bank and Charter One Bank" <clientservice.ref04388764.gps@citizensbank.com>', '"Josu" <josu@uninet.edu>']
['Citizens Bank and Charter One Bank Customer Service: Online Form Released! (mess_id: B5817899382D)', '"Citizens Bank 

In [50]:
for i in a.walk():
    print(i.get_content_type())

multipart/related
text/html
image/gif


In [63]:
for m in mailbox.mbox('emails-phishing.mbox'):
    print(m)

Return-Path: <identdep_op718@southtrust.com>
Delivered-To: user@example.com
Received: from example.com (example.com [192.168.2.0])
	by example.com (Postfix) with ESMTP id 791242A8D8
	for <user@example.com>; Thu, 16 Jun 2005 13:27:11 -0400 (EDT)
Received: by example.com (Example mail relay)
	id 3E693633C2; Thu, 16 Jun 2005 13:25:59 -0400 (EDT)
Delivered-To: user@example.com
Received: from 66-169-224-213.dhcp.mdfd.or.charter.com (66-169-224-213.dhcp.mdfd.or.charter.com [66.169.224.213])
	by example.com (Example mail relay) with SMTP id 24074633CF
	for <user@example.com>; Thu, 16 Jun 2005 17:25:58 +0000 (GMT)
FCC: mailbox://identdep_op718@southtrust.com/Sent
X-Identity-Key: id1
Date: Thu, 16 Jun 2005 21:19:57 +0300
From: SouthTrust <identdep_op718@southtrust.com>
X-Accept-Language: en-us, en
MIME-Version: 1.0
To: user@example.com
Subject: IMPORTANT ACCOUNT NOTICE FROM SOUTHTRUST BANK [Thu, 16 Jun 2005 12:20:57 -0600]
Content-Type: multipart/related;
 boundary="------------0402050208080506

Return-Path: <user@mail.example.com>
Delivered-To: user@example.com
Received: from mail.example.com (mail.example.com [192.168.6.4])
	by example.com (Postfix) with ESMTP id 87D652A88C
	for <user@example.com>; Sun,  7 Aug 2005 10:19:50 -0400 (EDT)
Received: by mail.example.com (Example mail relay)
	id 7CA4D6222F3; Sun,  7 Aug 2005 10:19:50 -0400 (EDT)
Delivered-To: user@example.com
Received: from mail.example.com (mail.example.com [192.168.3.6])
	by mail.example.com (Example mail relay) with ESMTP id 631FB6222F1
	for <user@example.com>; Sun,  7 Aug 2005 10:19:50 -0400 (EDT)
Received: (from user@localhost)
	by mail.example.com id KAA20822
	for user@example.com; Sun, 7 Aug 2005 10:19:50 -0400
Received: from boardermail.com ([60.31.59.236])
	by mail.example.com with SMTP id KAA20811
	for <user@example.com>; Sun, 7 Aug 2005 10:19:44 -0400
Received: from deutschebanc.de (mxpf01.ispgateway.de [80.67.18.4])
	by boardermail.com (Postfix) with ESMTP id DBB07B5E5E
	for <user@example.com>; Sat, 01

Return-Path: <user@mail.example.com>
Delivered-To: user@example.com
Received: from mail.example.com (mail.example.com [192.168.6.4])
	by example.com (Postfix) with ESMTP id EF5882A88C
	for <user@example.com>; Sun,  4 Sep 2005 13:49:13 -0400 (EDT)
Received: by mail.example.com (Example mail relay)
	id EAB38621C99; Sun,  4 Sep 2005 13:49:13 -0400 (EDT)
Delivered-To: user@example.com
Received: from mail.example.com (mail.example.com [192.168.3.6])
	by mail.example.com (Example mail relay) with ESMTP id BBDA8621C92
	for <user@example.com>; Sun,  4 Sep 2005 13:49:13 -0400 (EDT)
Received: (from user@localhost)
	by mail.example.com id NAA24088
	for user@example.com; Sun, 4 Sep 2005 13:49:13 -0400
Received: from 63-134-126-200.fibertel.com.ar (63-134-126-200.fibertel.com.ar [200.126.134.63])
	by mail.example.com with SMTP id NAA24079
	for <user@biocserver.cwru.edu>; Sun, 4 Sep 2005 13:49:10 -0400
Message-Id: <200509041749.NAA24079@mail.example.com>
FCC: mailbox://custservice_id_29753470320129@

UnicodeEncodeError: 'ascii' codec can't encode character '\ufffd' in position 2908: ordinal not in range(128)

In [55]:
for message in mailbox.mbox('emails-phishing.mbox').values():
    print(message)

Return-Path: <identdep_op718@southtrust.com>
Delivered-To: user@example.com
Received: from example.com (example.com [192.168.2.0])
	by example.com (Postfix) with ESMTP id 791242A8D8
	for <user@example.com>; Thu, 16 Jun 2005 13:27:11 -0400 (EDT)
Received: by example.com (Example mail relay)
	id 3E693633C2; Thu, 16 Jun 2005 13:25:59 -0400 (EDT)
Delivered-To: user@example.com
Received: from 66-169-224-213.dhcp.mdfd.or.charter.com (66-169-224-213.dhcp.mdfd.or.charter.com [66.169.224.213])
	by example.com (Example mail relay) with SMTP id 24074633CF
	for <user@example.com>; Thu, 16 Jun 2005 17:25:58 +0000 (GMT)
FCC: mailbox://identdep_op718@southtrust.com/Sent
X-Identity-Key: id1
Date: Thu, 16 Jun 2005 21:19:57 +0300
From: SouthTrust <identdep_op718@southtrust.com>
X-Accept-Language: en-us, en
MIME-Version: 1.0
To: user@example.com
Subject: IMPORTANT ACCOUNT NOTICE FROM SOUTHTRUST BANK [Thu, 16 Jun 2005 12:20:57 -0600]
Content-Type: multipart/related;
 boundary="------------0402050208080506

Return-Path: <user@mail.example.com>
Delivered-To: user@example.com
Received: from mail.example.com (mail.example.com [192.168.6.4])
	by example.com (Postfix) with ESMTP id 29E5E2A893
	for <user@example.com>; Tue, 23 Aug 2005 08:03:46 -0400 (EDT)
Received: by mail.example.com (Example mail relay)
	id 260C5621DDE; Tue, 23 Aug 2005 08:03:46 -0400 (EDT)
Delivered-To: user@example.com
Received: from mail.example.com (mail.example.com [192.168.3.6])
	by mail.example.com (Example mail relay) with ESMTP id 05FFA621DDD
	for <user@example.com>; Tue, 23 Aug 2005 08:03:46 -0400 (EDT)
Received: (from user@localhost)
	by mail.example.com id IAA00297
	for user@example.com; Tue, 23 Aug 2005 08:03:45 -0400
Received: from severone.actionlinesrl.it (ppp-217-133-5-133.cust-adsl.tiscali.it [217.133.5.133])
	by mail.example.com with ESMTP id IAA00288
	for <user@example.com>; Tue, 23 Aug 2005 08:03:36 -0400
Received: by severone.actionlinesrl.it (Postfix, from userid 1024)
	id 3062B65A32; Tue, 23 Aug 2005 0

UnicodeEncodeError: 'ascii' codec can't encode character '\ufffd' in position 2908: ordinal not in range(128)

In [84]:
train.head()

,id,subject,email,spam,all
0,0,Subject: A&L Daily to be auctioned in bankrupt...,URL: http://boingboing.net/#85534171\n Date: N...,0,Subject: A&L Daily to be auctioned in bankrupt...
1,1,"Subject: Wired: ""Stronger ties between ISPs an...",URL: http://scriptingnews.userland.com/backiss...,0,"Subject: Wired: ""Stronger ties between ISPs an..."
2,2,Subject: It's just too small ...,<HTML>\n <HEAD>\n </HEAD>\n <BODY>\n <FONT SIZ...,1,Subject: It's just too small ...
3,3,Subject: liberal defnitions\n,Depends on how much over spending vs. how much...,0,Subject: liberal defnitions\n Depends on how m...
4,4,Subject: RE: [ILUG] Newbie seeks advice - Suse...,hehe sorry but if you hit caps lock twice the ...,0,Subject: RE: [ILUG] Newbie seeks advice - Suse...


In [32]:
# 切分出一些验证集，分层抽样
from sklearn.model_selection import StratifiedShuffleSplit
splt = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=1)
for train_idx, valid_idx in splt.split(train, train['spam']):
    train_part = train.loc[train_idx]
    valid_part = train.loc[valid_idx]

y_train = train_part['spam']
y_valid = valid_part['spam']
X_train = train_part['all']
X_valid = valid_part['all']

X_test = test_csv['subject'] + ' ' + test_csv['email']
y_test = [0]*len(X_test) # 测试集没有标签，这么处理方便代码处理
y_test = torch.LongTensor(y_test) # 转成tensor

x_pred = ccac_test['all']
y_test = [0]*len(x_pred) # 测试集没有标签，这么处理方便代码处理
y_test = torch.LongTensor(y_test) # 转成tensor

In [33]:
# 下载预训练模型

In [34]:
!pip install transformers

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("./distilbert-base-uncased-finetuned-sst-2-english")
# distilbert-base-uncased-finetuned-sst-2-english

pretrain_model = AutoModelForSequenceClassification.from_pretrained("./distilbert-base-uncased-finetuned-sst-2-english")

In [6]:
a= "<html>,sdap:he@# got a dog.!"
token = tokenizer.tokenize(a)
print(token)
tid = tokenizer.convert_tokens_to_ids(token)
print(tid)

['<', 'html', '>', ',', 'sd', '##ap', ':', 'he', '@', '#', 'got', 'a', 'dog', '.', '!']
[1026, 16129, 1028, 1010, 17371, 9331, 1024, 2002, 1030, 1001, 2288, 1037, 3899, 1012, 999]


In [36]:
PAD, CLS = '[PAD]', '[CLS]'
max_seq_len = 128
bert_hidden = 768
num_classes = 2
learning_rate = 1e-5
decay = 0.01
num_epochs = 5
early_stop_time = 2000
batch_size = 32
save_path = "./best_model.ckpt" # 最好的模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [37]:
# 数据需要编码成 bert 需要的格式
# 需要 token_ids, attention_mask
def load_dataset(texts, labels):
    contents = []
    for t, label in zip(texts, labels):
        token = tokenizer.tokenize(t)
        token = [CLS] + token
        # ['[CLS]', 'subject', ':', 'cell', 'phones', 'coming', 'soon', '<', 'html', '>', ...]
        seq_len = len(token)
        mask = []
        token_ids = tokenizer.convert_tokens_to_ids(token)
        # [101, 3395, 1024, 3526, 11640, 2746, 2574, 1026, 16129, 。。。]
        if len(token) < max_seq_len: # 长度不够的，pad 补齐
            mask = [1]*len(token) + [0]*(max_seq_len-len(token))
            token_ids = token_ids + [0]*(max_seq_len-len(token))
        else: # 超长的，截断
            mask = [1]*max_seq_len
            token_ids = token_ids[:max_seq_len]
            seq_len = max_seq_len
        y = [0]*num_classes 
        y[label] = 1 # 处理下标签，方便后面计算 二元交叉熵损失
        contents.append((token_ids, y, seq_len, mask))
    return contents

In [38]:
# 编写数据集迭代器，训练的时候，每次取出 batch_size 个样本来更新权重
class datasetIter():
    def __init__(self, datasets, batch_size, device):
        self.datasets = datasets
        self.idx = 0
        self.device = device
        self.batch_size = batch_size
        self.batches = len(datasets)//batch_size
        self.residues = False
        if len(datasets)%batch_size != 0:
            self.residues = True # 剩余不足 batch_size 个的样本
    def __next__(self):
        if self.residues and self.idx==self.batches:
            batch_data = self.datasets[self.idx * self.batch_size : len(self.datasets)]
            self.idx += 1
            batch_data = self._to_tensor(batch_data)
            return batch_data
        elif self.idx > self.batches:
            self.idx = 0
            raise StopIteration
        else:
            batch_data = self.datasets[self.idx * self.batch_size : (self.idx+1) * self.batch_size]
            self.idx += 1
            batch_data = self._to_tensor(batch_data)
            return batch_data
    def _to_tensor(self, datasets):
        x = torch.LongTensor([item[0] for item in datasets]).to(self.device)
        y = torch.FloatTensor([item[1] for item in datasets]).to(self.device)
        seq_len = torch.LongTensor([item[2] for item in datasets]).to(self.device)
        mask = torch.LongTensor([item[3] for item in datasets]).to(self.device)
        return (x, seq_len, mask), y
    def __iter__(self):
        return self
    def __len__(self):
        if self.residues:
            return self.batches + 1
        else:
            return self.batches

In [39]:
def build_iter(datasets, batch_size, device):
    iter = datasetIter(datasets,batch_size,device)
    return iter

In [40]:
# 4. 定义模型

In [41]:
class myModel(nn.Module):
    def __init__(self):
        super(myModel, self).__init__()
        self.pretrain_model = pretrain_model # 预训练的bert模型
        for param in self.pretrain_model.parameters():
            param.requires_grad = True # 打开 finetune 开关
    def forward(self, x):
        context = x[0]
        mask = x[2]
        out = self.pretrain_model(context, attention_mask=mask)
        out = torch.sigmoid(out.logits) # sigmoid到 (0,1) 方便计算交叉熵
        return out

In [42]:
# 5. 训练
import time
import torch.nn.functional as F

from sklearn import metrics
from transformers.optimization import AdamW

In [43]:
# 辅助计时函数
def get_time_dif(starttime):
    # calculate used time
    endtime = time.time()
    return timedelta(seconds=int(round(endtime-starttime)))

In [44]:
# 训练
def train(model, train_iter, dev_iter, test_iter):
    starttime = time.time() # 记录开始时间
    model.train()
    optimizer = AdamW(model.parameters(),lr=learning_rate,weight_decay=decay)
    total_batch = 0
    dev_best_loss = float("inf")
    last_improve = 0
    no_improve_flag = False
    model.train()
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch+1, num_epochs))
        for i, (X, y) in enumerate(train_iter):
            outputs = model(X) # batch_size * num_classes
            model.zero_grad() # 清理梯度增量
            loss = F.binary_cross_entropy(outputs, y)
            loss.backward()
            optimizer.step()
            if total_batch%100 == 0: # 打印训练信息
                truelabels = torch.max(y.data, 1)[1].cpu()
                pred = torch.max(outputs, 1)[1].cpu()
                train_acc = metrics.accuracy_score(truelabels, pred)
                # 调用 评估函数 检查验证集上的效果
                dev_acc, dev_loss = evaluate(model, dev_iter) 
                # 检查验证集上的效果， 保留效果最好的
                if dev_loss < dev_best_loss:
                    dev_best_loss = dev_loss
                    torch.save(model.state_dict(), save_path)
                    improve = '*'
                    last_improve = total_batch
                else:
                    improve = ' '
                time_dif = get_time_dif(starttime)
                # 打印训练信息，id : >右对齐，n 宽度，.3 小数位数
                msg = 'Iter:{0:>6}, Train Loss:{1:>5.2}, Train Acc:{2:>6.2}, Val Loss:{3:>5.2}, val Acc :{4:>6.2%}, Time:{5} {6}'
                print(msg.format(total_batch, loss.item(),train_acc, dev_loss, dev_acc, time_dif, improve))
                model.train()
            total_batch += 1
            # 如果长时间没有改进，认为收敛，停止训练
            if total_batch - last_improve > early_stop_time:
                print("no improve after {} times, stop!".format(early_stop_time))
                no_improve_flag = True
                break
        if no_improve_flag:
            break
    # 调用 测试函数，生成预测结果
    test(model, test_iter)

In [45]:
# 评估函数
def evaluate(model, dev_iter):
    model.eval() # 评估模式
    loss_total = 0
    pred_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad(): # 不记录图的操作，不更新梯度
        for X, y in dev_iter:
            outputs = model(X)
            loss = F.binary_cross_entropy(outputs, y)
            loss_total += loss
            truelabels = torch.max(y.data, 1)[1].cpu()
            pred = torch.max(outputs, 1)[1].cpu().numpy()
            labels_all = np.append(labels_all, truelabels)
            pred_all = np.append(pred_all, pred)
    acc = metrics.accuracy_score(labels_all, pred_all)
    return acc, loss_total/len(dev_iter)

In [46]:
# 测试函数
def test(model, test_iter):
    model.load_state_dict(torch.load(save_path)) # 加载最佳模型
    model.eval() # 评估模式
    pred_all = np.array([], dtype=int)
    with torch.no_grad():
        for X, y in test_iter:
            outputs = model(X)
            pred = torch.max(outputs, 1)[1].cpu().numpy()
            pred_all = np.append(pred_all, pred)
    # 写入提交文件
    id = test_csv['id']
    output = pd.DataFrame({'id':id, 'Class': pred_all})
    output.to_csv("submission_bert.csv",  index=False)

In [47]:
# 运行主程序
# 确定随机数
np.random.seed(520)
torch.manual_seed(520)
torch.cuda.manual_seed_all(520)
torch.backends.cudnn.deterministic = True

# 加载数据
train_data = load_dataset(X_train, y_train)
valid_data = load_dataset(X_valid, y_valid)
test_data = load_dataset(X_test, y_test)

# 数据迭代器
train_iter = build_iter(train_data, batch_size, device)
valid_iter = build_iter(valid_data, batch_size, device)
test_iter = build_iter(test_data, batch_size, device)

# 模型
model = myModel().to(device)

# 训练、评估、测试
train(model, train_iter, valid_iter, test_iter)

Token indices sequence length is longer than the specified maximum sequence length for this model (7508 > 512). Running this sequence through the model will result in indexing errors
C:\Users\Jason\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5
Iter:     0, Train Loss:  1.3, Train Acc:  0.59, Val Loss: 0.71, val Acc :73.71%, Time:0:03:43 *


KeyboardInterrupt: 

In [ ]:
predict_x = model.predict(
)
y_pred=np.argmax(predict_x,axis=1)
y_pred[:9]